<a href="https://colab.research.google.com/github/has-c/UserMobilityMining/blob/master/Gowalla%20ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, Softmax
from sklearn.metrics import accuracy_score
import datetime

#disable warnings
import warnings
warnings.filterwarnings("ignore")

## Helper Functions

In [47]:
def alignPastCurrentPOI(checkins, dateList):

    #group by userId, then checkin date and then order by time 
    uniqueUserIds = np.unique(checkins['userid'])
    uniqueCheckinDates = sorted(np.unique(dateList))

    sortedCheckins = pd.DataFrame([])

    for userId in uniqueUserIds:
        for checkinDate in uniqueCheckinDates:
            
            #group by userId and checkin date 
            groupedByUserId = checkins[userId==checkins['userid']]
            groupedByCheckinDate = groupedByUserId[groupedByUserId['Date'] == checkinDate]
            if groupedByCheckinDate.shape[0] == 0:
                continue
                
            #sort by time 
            groupedByCheckinDate.sort_values(by ='Time',ascending=True, inplace=True)
            groupedByCheckinDate.index = range(0, groupedByCheckinDate.shape[0])
            
            #create list of sorted checkins 
            sortedCheckins = pd.concat([sortedCheckins, groupedByCheckinDate], axis=0, ignore_index=True)
    
    #align past and current checkins - features and labels entire df
    pastCurrentCheckins = pd.DataFrame([])

    for userId in uniqueUserIds:
        
        #group by userId
        groupedByUserId = sortedCheckins[userId==checkins['userid']]
        
        for rowIndex in groupedByUserId.index:
            try:
                pastPOI = groupedByUserId.loc[rowIndex]
                currentPOI = groupedByUserId.loc[rowIndex+1]
                poiInstance = list(currentPOI) + list(pastPOI)
                pastCurrentCheckins = pd.concat([pastCurrentCheckins, pd.DataFrame(poiInstance).T], axis=0)
            except KeyError:
                break

    #rename fields 
    pastCurrentCheckinDfColumnNames = ['Current ' + name for name in sortedCheckins.columns] + ['Past ' + name for name in sortedCheckins.columns]
    pastCurrentCheckins.columns = pastCurrentCheckinDfColumnNames

    return pastCurrentCheckins

### Preprocessing 
<ul>
    <li> One hot encoding </li>
    <li> Standard scaling </li>
</ul>

In [81]:
#import data
checkinsDf = pd.read_csv("/home/usermobilitymining/Notebooks/volume/Hasnain/Processed Data/processedCheckins.csv")
nzCheckins = pd.read_csv("/home/usermobilitymining/Notebooks/volume/Hasnain/Processed Data/nzCheckinsWithGridTokens.csv")

In [82]:
#join checkins df and nz checkins to only extract NZ checkins 
checkinsDf['id'] = checkinsDf.index #create id field
nzCheckins.drop("lat", axis=1, inplace=True) #drop lat so no error
nzCheckins.drop("lng", axis=1, inplace=True) #drop lng so no error
checkinsDf = nzCheckins.join(checkinsDf.set_index('id'), on='id')
checkinsDf.drop("Unnamed: 0", axis=1, inplace=True)

In [83]:
#convert date and time to datetime variables
#extract day of the week and hour of checkin
checkinDate = [datetime.datetime.strptime(date, "%Y-%m-%d").date() for date in checkinsDf['Date']]
checkinDay = [date.weekday() for date in checkinDate]
checkinTime = [datetime.datetime.strptime(time, '%H:%M:%S').time() for time in checkinsDf['Time']]
checkinHour = [time.hour for time in checkinTime]

In [84]:
#one hot encode the time variable
ohe = OneHotEncoder(dtype=np.int8, n_values=24)
hourOfCheckinEncoded = ohe.fit_transform(np.array(checkinHour).reshape(-1,1))
timeColumns = ['Time ' + str(num) for num in np.arange(0,24)]
timeDf = pd.DataFrame(hourOfCheckinEncoded.toarray(), columns=timeColumns)

#one hot encode the day of the week variable
ohe = OneHotEncoder(dtype=np.int8, n_values=7)
dayOfTheWeekEncoded = ohe.fit_transform(np.array(checkinDay).reshape(-1,1))
dayColumns = ['DayOfWeek ' + str(num) for num in np.arange(0,7)]
dayDf = pd.DataFrame(dayOfTheWeekEncoded.toarray(), columns=dayColumns)

#one hot encode the main category
numberOfMainCategories = len(np.unique(checkinsDf['Main Category']))
labelEncoder = LabelEncoder()
mainCategoriesEncoded = labelEncoder.fit_transform(checkinsDf['Main Category'].values.reshape(-1,1))
ohe = OneHotEncoder(dtype=np.int8, n_values=numberOfMainCategories)
mainCategoriesEncoded = ohe.fit_transform(labelEncodedCategories.reshape(-1,1))
categoryDf = pd.DataFrame(mainCategoriesEncoded.toarray(), columns=labelEncoder.classes_)

In [85]:
#concat fields
checkinsDf = pd.concat([checkinsDf, timeDf], axis=1,copy=False) 
checkinsDf = pd.concat([checkinsDf, dayDf], axis=1,copy=False) 
checkinsDf = pd.concat([checkinsDf, categoryDf], axis=1,copy=False) 

In [86]:
#drop fields 
checkinsDf.drop("Date", axis=1, inplace=True)
checkinsDf.drop("Time", axis=1, inplace=True)
checkinsDf.drop("Main Category", axis=1, inplace=True)
checkinsDf.drop("Locations", axis=1, inplace=True)

In [90]:
#train test split - normal 
labelDf = checkinsDf[['Community','Entertainment','Food','Nightlife','Outdoors','Shopping','Travel']]
featuresDf = checkinsDf.drop(['Community','Entertainment','Food','Nightlife','Outdoors','Shopping','Travel'], axis=1)

#train test split
featuresTrain, featuresTest, labelTrain, labelTest = train_test_split(featuresDf, labelDf, test_size=0.2, random_state=42)

### Neural Net

In [95]:
model = Sequential()
model.add(Dense(35, input_dim=45, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [96]:
#fit model 
history = model.fit(featuresTrain, labelTrain, epochs=100)

Epoch 1/100
  256/16160 [..............................] - ETA: 39s - loss: 3194817.9062 - accuracy: 0.0859 

ValueError: could not convert string to float: 'a'

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int8)

In [ ]:
#predict
labelPredicted = model.predict(featuresTest)